In [1]:
import os
os.getcwd()

'C:\\Users\\USER\\Face mask detection'

In [11]:
'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''
TrainingImagePath = 'D:\\Hashir\\IVY\\Python\\Project\\face mask detection\Train'

from keras.preprocessing.image import ImageDataGenerator

# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


{'with mask': 0, 'without mask': 1}

### Creating a list of faces and classes

In [12]:
Trainclasses=training_set.class_indices

Resultmap={}
for facevalue, facename in zip(Trainclasses.values() , Trainclasses.keys()):
    Resultmap[facevalue]=facename
    
import pickle 
with open('D:\\Hashir\\IVY\\Python\\Project\\face mask detection\\Resultmap.pkl', 'wb') as f:
    pickle.dump(Resultmap, f, pickle.HIGHEST_PROTOCOL)
    
print('Mapping masks and its Type :', Resultmap)

Outputneurons=len(Resultmap)
print('\n No.of. neurons:', Outputneurons)

Mapping masks and its Type : {0: 'with mask', 1: 'without mask'}

 No.of. neurons: 2


### Creating the CNN model

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

classifier= Sequential()

classifier.add(Convolution2D(32, kernel_size=(5,5), strides=(1,1), input_shape=(64,64,3), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
#adding another layer for accuracy
classifier.add(Convolution2D(32, kernel_size=(5,5), strides=(1,1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

classifier.add(Flatten())
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(Outputneurons, activation='softmax'))

classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


### Fitting the mode

In [14]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=5,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=5)

EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Epoch 1/10
5/5 [==============================] - 1s 286ms/step - loss: 0.6626 - accuracy: 0.6324 - val_loss: 0.5720 - val_accuracy: 0.6438
Epoch 2/10
5/5 [==============================] - 1s 220ms/step - loss: 0.4460 - accuracy: 0.7937 - val_loss: 0.2897 - val_accuracy: 0.9125
Epoch 3/10
5/5 [==============================] - 1s 222ms/step - loss: 0.2940 - accuracy: 0.8938 - val_loss: 0.1807 - val_accuracy: 0.9375
Epoch 4/10
5/5 [==============================] - 1s 197ms/step - loss: 0.1773 - accuracy: 0.9191 - val_loss: 0.1092 - val_accuracy: 0.9500
Epoch 5/10
5/5 [==============================] - 1s 223ms/step - loss: 0.1183 - accuracy: 0.9412 - val_loss: 0.1204 - val_accuracy: 0.9500
Epoch 6/10
5/5 [==============================] - 1s 211ms/step - loss: 0.1697 - accuracy: 0.9338 - val_loss: 0.1012 - val_accuracy: 0.9625
Epoch 7/10
5/5 [==============================] - 1s 216ms/step - loss: 0.0667 - accuracy: 0.9750 - val_loss: 0.1091 - val_accuracy: 0.9500
Epoch 8/10
5/5 [====

### Saving the model

In [15]:
## Saving the model
classifier.save('D:\\Hashir\\IVY\\Python\\Project\\face mask detection\\classifier.pkl')

INFO:tensorflow:Assets written to: D:\Hashir\IVY\Python\Project\face mask detection\classifier.pkl\assets


### Testing the model on a different face

In [20]:
# with_mask

import numpy as np
from keras.preprocessing import image

testImage="D:\\Hashir\\IVY\\Python\\Project\\face mask detection\\Test\\with mask\\123-with-mask.jpg"
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',Resultmap[np.argmax(result)])

########################################
Prediction is:  with mask


In [21]:
# without_mask

import numpy as np
from keras.preprocessing import image

testImage="D:\\Hashir\\IVY\\Python\\Project\\face mask detection\\Test\\without mask\\augmented_image_137.jpg"
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',Resultmap[np.argmax(result)])

########################################
Prediction is:  without mask
